In [1]:
import numpy as np
import pandas as pd

In [2]:
texts_month = pd.read_csv('data/texts_31days.csv', index_col=0)
texts_first = pd.read_csv('data/texts_12.04-13.04.csv', index_col=0)
texts_second = pd.read_csv('data/texts_13.04-14.04.csv', index_col=0)
texts = pd.concat([texts_month, texts_first, texts_second])
texts.index.names = ['url_id']

texts.to_csv('data/texts.csv')

In [17]:
from datetime import datetime
from datetime import timezone
from zipfile import ZipFile
import os

def make_df(start_time, end_time):
    timestamps = sorted(os.listdir('data/timestamps'))
    start_timestamp, end_timestamp = make_timestamps_from_datetime(start_time, end_time, timestamps)
    interval = make_interval(start_timestamp, end_timestamp, timestamps)
    with ZipFile('data/timestamps.zip') as timestamps_zip:
        df_list = [pd.read_csv(timestamps_zip.open("timestamps/" + file), header=None, names=['fullVisitorId', 'url_id', 'visitStartTime']) for file in interval]
    df = pd.concat(df_list)
    labels, levels = pd.factorize(df['fullVisitorId'])
    df['user_id'] = labels
    df.set_index(['user_id','url_id'], inplace=True, drop=True)
    return df

# first = 12/03/2017 07:00:00, last = 14/04/2017 11:11:29 1491818423 1491991225
def make_timestamps_from_datetime(start_time, end_time, timestamps):
    if start_time == 'first':
        start_timestamp = timestamps[0]
    else:
        start_datetime = datetime.strptime(start_time, '%d/%m/%Y %H:%M:%S')
        start_timestamp = (start_datetime - datetime(1970, 1, 1)).total_seconds()
        
    if end_time == 'last':
        end_timestamp = timestamps[-1]
    else:
        end_datetime = datetime.strptime(end_time, '%d/%m/%Y %H:%M:%S')
        end_timestamp = (end_datetime - datetime(1970, 1, 1)).total_seconds()
    return (start_timestamp, end_timestamp)

def make_interval(start_timestamp, end_timestamp, timestamps):
    start_timestamp = str(start_timestamp)
    end_timestamp = str(end_timestamp)
    interval = [t for t in timestamps if t >= start_timestamp and t <= end_timestamp]
    return interval

# Using texts.csv to make urls for each url_id
def make_urls_df():
    texts = pd.read_csv('data/texts.csv')
    tag_cleaned = texts['tag'].str.split().str.get(0)
    texts['tag_cleaned'] = tag_cleaned
    texts['url_id'] = texts['url_id'].astype(str)
    texts['pagePath'] = '/t/' + texts['tag_cleaned'] + '/' + texts['url_id']
    texts.set_index(['url_id'], inplace=True)
    urls = texts.drop(['subtitle', 'tag', 'title', 'tag_cleaned'], axis=1)
    return urls

def merge_df(df, urls):
    df.reset_index(level=['url_id'], inplace=True)
    df.reset_index(level=['user_id'], inplace=True)
    urls.reset_index(level=['url_id'], inplace=True)
    urls['url_id'] = urls['url_id'].astype(int)
    df['fullVisitorId'] = df['fullVisitorId'].astype(str)
    df_result = pd.merge(df, urls, on='url_id', how='left')
    labels, levels = pd.factorize(df_result['url_id'])
    df_result['url_id'] = labels
    df_result.set_index(['user_id', 'url_id'], inplace=True)
    df_result.sort_index(inplace=True)
    return df_result

df = make_df('15/03/2017 10:00:00', '16/03/2017 10:00:00')
urls = make_urls_df()
df_result = merge_df(df, urls)

In [18]:
display(df_result.info())
display(df_result.head(20))
display(df_result.tail(20))

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 11969533 entries, (0, 0) to (1014978, 40773)
Data columns (total 3 columns):
fullVisitorId     object
visitStartTime    int64
pagePath          object
dtypes: int64(1), object(2)
memory usage: 373.4+ MB


None

fullVisitorId  visitStartTime                pagePath
user_id url_id                                                              
0       0        1036536157958312061      1489571954     /t/экономика/939611
        0        1036536157958312061      1489571954     /t/экономика/939611
1       1       -4611953592302795458      1489571994       /t/новости/163224
        1       -4611953592302795458      1489571994       /t/новости/163224
        1       -4611953592302795458      1489571994       /t/новости/163224
        1       -4611953592302795458      1489572378       /t/новости/163224
        1       -4611953592302795458      1489572378       /t/новости/163224
        1       -4611953592302795458      1489572378       /t/новости/163224
        292     -4611953592302795458      1489571990  /t/происшествия/879205
        460     -4611953592302795458      1489571970     /t/подростки/972878
        460     -4611953592302795458      1489571970         /t/семья/972878
        460     -4611953592302795458      1489571970         /t/семья/972878
        2041    -4611953592302795458      1489572411          /t/суды/893693
2       2        5259739520833585814      1489572000  /t/происшествия/985017
        2        5259739520833585814      1489572000       /t/новости/985017
        2        5259739520833585814      1489572000       /t/новости/985017
        39       5259739520833585814      1489572710     /t/школьники/985295
        39       5259739520833585814      1489572710       /t/новости/985295
        39       5259739520833585814      1489572710       /t/новости/985295
        64       5259739520833585814      1489572000       /t/украина/985010

fullVisitorId  visitStartTime                  pagePath
user_id url_id                                                                
1014970 43125   -6816835129062106908      1489658326  /t/беллапотемкина/985934
1014971 43125   -7051498492872711905      1489658314     /t/евровидение/985934
        43125   -7051498492872711905      1489658314  /t/беллапотемкина/985934
        43125   -7051498492872711905      1489658314  /t/беллапотемкина/985934
1014972 1290     4002436324377211799      1489658341        /t/культура/984990
        1290     4002436324377211799      1489658341        /t/культура/984990
1014973 1290     2092310398366814739      1489658289        /t/культура/984990
        1290     2092310398366814739      1489658289        /t/культура/984990
1014974 1290    -1500186809021425371      1489658318        /t/культура/984990
        1290    -1500186809021425371      1489658318        /t/культура/984990
1014975 38490    6970252073615751344      1489658301      /t/технологии/985895
        38490    6970252073615751344      1489658301      /t/технологии/985895
        38490    6970252073615751344      1489658301      /t/технологии/985895
1014976 942      1113520981978935195      1489658327      /t/технологии/984686
1014977 43205    4754752333300011806      1489658289      /t/референдум/986011
        43205    4754752333300011806      1489658289         /t/новости/986011
        43205    4754752333300011806      1489658289         /t/новости/986011
1014978 40773   -6738271101230342151      1489658288            /t/фото/921137
        40773   -6738271101230342151      1489658288         /t/новости/921137
        40773   -6738271101230342151      1489658288         /t/новости/921137